In [2]:
import PyPDF2
import os
from os import listdir
from os.path import isfile,join
from io import StringIO
import pandas as pd
from collections import Counter
import en_core_web_sm
nlp=en_core_web_sm.load()
from spacy.matcher import PhraseMatcher
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO 
import re
import pandas
from pandas import ExcelWriter
import datetime
from tkinter import filedialog
from tkinter import Button, Tk, HORIZONTAL
from tkinter.ttk import Progressbar
import time
import threading

In [3]:
#optional parameter PAGES can restrict which pages to process
def convert_pdf_to_txt(path, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)
    
    #Instantiate the PDFminer objects
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)
    
    infile = open(path, 'rb') #open the file for read in binary mode
    
    for page in PDFPage.get_pages(infile, pagenums):
        #iterate with the pdf interpreter through the pdf pages
        interpreter.process_page(page)
    
    #Close the files and converters
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close()
    text=str(text)
    text=text.replace("\\n"," ")
    text=text.replace("\n"," ")
    text=text.replace(" \x0c "," ")
    text=text.lower()
    return text   #return the text as str

def removeques(text):
    text=re.sub(r"\bq1. why should you be hired for this internship?\b"," " ,text)
    text=re.sub(r"\bq4. which is your strongest open source tool among sas, r, tableau, python?\b"," " ,text)
    text=re.sub(r"\bq3. how will you rate yourself on sql knowledge?\b"," " ,text)
    return text
data=pd.read_csv('datascience_ontology.csv')
def createProfile(file):
    text=convert_pdf_to_txt(file)
    text=removeques(text)
    #print(text)
    keyword_dict=pd.read_csv("datascience_ontology.csv")
    
    time_word=[nlp(text) for text in keyword_dict["TimeSeries"].dropna(axis=0)]
    nlp_word=[nlp(text) for text in keyword_dict["NLP"].dropna(axis=0)]
    ml_word=[nlp(text) for text in keyword_dict["Machine Learning"].dropna(axis=0)]
    python_word=[nlp(text) for text in keyword_dict["Python"].dropna(axis=0)]
    r_word=[nlp(text) for text in keyword_dict["R"].dropna(axis=0)]
    dl_word=[nlp(text) for text in keyword_dict["Deep Learning"].dropna(axis=0)]
    BI_word=[nlp(text) for text in keyword_dict["BI Tools"].dropna(axis=0)]
    dataeng_word=[nlp(text) for text in keyword_dict["Data Engineering"].dropna(axis=0)]
    big_word=[nlp(text) for text in keyword_dict["BigData"].dropna(axis=0)]
    
    matcher=PhraseMatcher(nlp.vocab)
    matcher.add('TimeSeries',None,*time_word)
    matcher.add('BI_Tools',None,*BI_word)
    matcher.add('R',None,*r_word)
    matcher.add('NLP',None,*nlp_word)
    matcher.add('Bigdata',None,*big_word)
    matcher.add('Machine_learning',None,*ml_word)
    matcher.add('Python',None,*python_word)
    matcher.add('DeepLearning',None,*dl_word)
    matcher.add('DataEngineering',None,*dataeng_word)
    doc=nlp(text)
    
    d=[]
    matches=matcher(doc)
    for match_id,start,end in matches:
        rule_id=nlp.vocab.strings[match_id]
        span=doc[start:end]
        d.append((rule_id,span.text))
    keywords="\n".join(f'{i[0]} {i[1]} ({j})' for i,j in Counter(d).items())
    
    df=pd.read_csv(StringIO(keywords),names=['Keywords_list'])
    
    df1=pd.DataFrame(df.Keywords_list.str.split(' ',1).tolist(),columns=['Subject','Keyword'])
    df2=pd.DataFrame(df1.Keyword.str.split('(',1).tolist(),columns=['Keyword','Count'])
    df3=pd.concat([df1['Subject'],df2['Keyword'],df2['Count']],axis=1)
    df3['Count']=df3['Count'].apply(lambda x:x.rstrip(")"))
    #print(df)
    #print(df1)
    #print(df2)
    #print(df3)
    base=os.path.basename(file)
    filename=os.path.splitext(base)[0]
    name = filename.split('_')
    name2 = name[0]
    name2 = name2.lower()
    ## converting str to dataframe
    name3 = pd.read_csv(StringIO(name2),names = ['Candidate Name'])
    
    dataf = pd.concat([name3['Candidate Name'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
    dataf['Candidate Name'].fillna(dataf['Candidate Name'].iloc[0], inplace = True)
    #return text
    return(dataf)
def get_report():
    #mypath=folder
    #onlyfiles=[os.path.join(mypath,f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath,f))]
    #print("success")
    currentDT = datetime.datetime.now()
    currentDT=currentDT.strftime("%Y%m%d%H%M%S")
    final_database=pd.DataFrame()
    i = 0 

    while i < len(onlyfiles):
        file = onlyfiles[i]
        dat = createProfile(file)
        final_database = final_database.append(dat)
        i +=1
    final_database2 = final_database['Keyword'].groupby([final_database['Candidate Name'], final_database['Subject']]).count().unstack()
    final_database2.reset_index(inplace = True)
    final_database2.fillna(0,inplace=True)
    #print(final_database2)
    writer = ExcelWriter("CandidateSkillExcel"+str(currentDT)+".xlsx")
    final_database2.to_excel(writer,'resume_skill_Sheet')
    writer.save()
    #final_database2 = final_database['Keyword'].groupby([final_database['Candidate Name'], final_database['Subject']]).count().unstack()
    #final_database2.reset_index(inplace = True)
    #final_database2.fillna(0,inplace=True)
    new_data = final_database2.iloc[:,1:]
    new_data.index = final_database2['Candidate Name']
    #execute the below line if you want to see the candidate profile in a csv format
    sample2=new_data.to_csv("CandidateSkillSaved"+str(currentDT)+".csv")
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size': 10})
    ax = new_data.plot.barh(title="Resume keywords by category", legend=False, figsize=(30,17), stacked=True)
    labels = []
    for j in new_data.columns:
        for i in new_data.index:
            label = str(j)+": " + str(new_data.loc[i][j])
            labels.append(label)
    patches = ax.patches
    for label, rect in zip(labels, patches):
        width = rect.get_width()
        if width > 0:
            x = rect.get_x()
            y = rect.get_y()
            height = rect.get_height()
            ax.text(x + width/2., y + height/2., label, ha='center', va='center')
    plt.savefig("Resume_Statistic"+str(currentDT)+".png")
    #print("Saved sucessfully")
    return "success"
#final_database

In [4]:
currentDT = datetime.datetime.now()

print(currentDT)
currentDT.strftime("%Y%m%d%H%M%S")

2019-03-04 15:56:41.942265


'20190304155641'

In [9]:
from tkinter import *
from tkinter.ttk import *

master = Tk()
master.geometry("500x200")
Label(master, text="Selected Folder is:").grid(row=1)
def myfunc():
    s=str(btn['state'])
    print(s)
    inputValue="Loading..."
    txt=get_report()
    print(s)
    if s == "active":
        label = Label(master, text="Loading...").grid(row=3)
    
    #if txt=="success":
    #    label = Label(master, text="Action completed!Reports generated successfully").grid(row=2)
btn = Button(master, text="Get Report", command=myfunc)

e1 = Entry(master,width=50)
#e2 = Entry(master)
folder_selected = filedialog.askdirectory()
fold=str(folder_selected)

e1.insert(0, fold)
#print(btn.state)
e1.grid(row=1, column=1)
btn.grid(row=1,column=3)
#e2.grid(row=1, column=1)
mypath=folder_selected
onlyfiles=[os.path.join(mypath,f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath,f))]

mainloop( )

normal
normal
